# Import

In [1]:
import os, os.path
import pandas as pd
import numpy as np, os 
from tqdm import tqdm
from collections import Counter

from torch.utils.data import Dataset, DataLoader
import torch


# Access to file

In [2]:
list_txt = os.listdir("train") # dir is your directory path
number_files = len(list_txt)
number_files

15594

In [3]:
type(list_txt)

list

In [4]:
index_txt = list_txt.index('0A0AA9C21C5D.txt')
index_txt                   

6804

In [5]:
os.getcwd()

'/home/arno/Documents/1_Study/Project/NLP_sentiment_pred'

In [6]:
def open_text(text):
    with open('train/'+text+'.txt') as f:
        text = f.read()
    # print out the first 100 characters
    return text

# NLP Solution

In [7]:
train_csv = pd.read_csv('train.csv')

In [8]:
print(train_csv.shape )
train_csv.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [9]:
train_csv.iloc[0]

id                                                         423A1CA112E2
discourse_id                                            1622627660524.0
discourse_start                                                     8.0
discourse_end                                                     229.0
discourse_text        Modern humans today are always on their phone....
discourse_type                                                     Lead
discourse_type_num                                               Lead 1
predictionstring      1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
Name: 0, dtype: object

In [16]:
test_names, test_texts = [], []
for f in os.listdir('test'):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts.head()

,id,text
0,D46BCB48440A,"When people ask for advice,they sometimes talk..."
1,18409261F5C2,80% of Americans believe seeking multiple opin...
2,D72CB1C11673,Making choices in life can be very difficult. ...
3,DF920E0A7337,Have you ever asked more than one person for h...
4,0FB0700DAF44,"During a group project, have you ever asked a ..."


# Train CSV review and creation of counters


In [187]:
def creation_count(train_csv):
    # list of counters
    lead_counts = Counter()
    position_counts = Counter()
    claim_counts = Counter()
    counterclaim_counts = Counter()
    rebuttal_counts = Counter()
    evidence_counts = Counter()
    conclustate_counts = Counter()

    for i in range(len(train_csv.discourse_id)):
        text = train_csv.discourse_text[i]
        text = text.split(' ')
        if train_csv.discourse_type[i] == 'Lead':
            for words_ind in range(len(text)):
                lead_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Position':
            for words_ind in range(len(text)):
                position_counts[text[words_ind-1]] += 1
            
        elif train_csv.discourse_type[i] == 'Claim':
            for words_ind in range(len(text)):
                claim_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Counterclaim':
            for words_ind in range(len(text)):
                counterclaim_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Rebuttal':
            for words_ind in range(len(text)):
                rebuttal_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Evidence':
            for words_ind in range(len(text)):
                evidence_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Concluding Statement':
            for words_ind in range(len(text)):
                conclustate_counts[text[words_ind-1]] += 1
    
    return lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts


In [188]:
lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts = creation_count(train_csv)

In [208]:
def counter_ratio (train_csv):
    lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts = creation_count(train_csv)
    total_count = lead_counts+position_counts+claim_counts+counterclaim_counts+rebuttal_counts+evidence_counts+conclustate_counts 
    # lead count
    lead_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            lead_ratio = lead_counts[term] / float(total_count[term]+1)
            lead_counts_ratios[term] = lead_ratio
            
    # position count
    position_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            position_ratio = position_counts[term] / float(total_count[term]+1)
            position_counts_ratios[term] = position_ratio
            
    #claim count
    claim_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            claim_ratio = claim_counts[term] / float(total_count[term]+1)
            claim_counts_ratios[term] = claim_ratio
            
    #counter claim count
    counterclaim_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            counterclaim_ratio = counterclaim_counts[term] / float(total_count[term]+1)
            counterclaim_counts_ratios[term] = counterclaim_ratio
            
    #rebuttal count
    rebuttal_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            rebuttal_ratio = rebuttal_counts[term] / float(total_count[term]+1)
            rebuttal_counts_ratios[term] = rebuttal_ratio
            
    #evidence count
    evidence_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            evidence_ratio = evidence_counts[term] / float(total_count[term]+1)
            evidence_counts_ratios[term] = evidence_ratio
            
    #concluding statement count
    conclustate_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            conclustate_ratio = conclustate_counts[term] / float(total_count[term]+1)
            conclustate_counts_ratios[term] = conclustate_ratio
    
    return lead_counts_ratios, position_counts_ratios, claim_counts_ratios, counterclaim_counts_ratios,rebuttal_counts_ratios, evidence_counts_ratios, conclustate_counts_ratios

In [209]:
lead_counts_ratios, position_counts_ratios, claim_counts_ratios, counterclaim_counts_ratios,rebuttal_counts_ratios, evidence_counts_ratios, conclustate_counts_ratios = counter_ratio(train_csv)

In [210]:
lead_counts_ratios.most_common()

[('wondered', 0.8557692307692307),
 ('advice?', 0.816793893129771),
 ('Have', 0.7612551159618008),
 ('activity?', 0.7571428571428571),
 ('home?', 0.696969696969697),
 ('Should', 0.6943231441048034),
 ('choice?', 0.6779661016949152),
 ('"Making', 0.6357142857142857),
 ('conferencing.', 0.6050420168067226),
 ('school?', 0.5603448275862069),
 ('Do', 0.5449664429530201),
 ('Mars?', 0.5370370370370371),
 ('Smile"', 0.527027027027027),
 ('circling', 0.5238095238095238),
 ('debate', 0.5152838427947598),
 ('Ever', 0.5151515151515151),
 ('Coming"', 0.5),
 ('assure', 0.49382716049382713),
 ('controversial', 0.49230769230769234),
 ('Did', 0.49056603773584906),
 ('driving?', 0.4791666666666667),
 ('begun', 0.4722222222222222),
 ('people?', 0.47126436781609193),
 ('Nick', 0.46835443037974683),
 ('Recently,', 0.4528301886792453),
 ('"Driverless', 0.44878048780487806),
 ('Today', 0.44537815126050423),
 ('enables', 0.44525547445255476),
 ('likeness', 0.4426229508196721),
 ('Does', 0.4375),
 ('Everyday

In [211]:
position_counts_ratios.most_common()

[('Dear', 0.5769230769230769),
 ('suggests', 0.4972375690607735),
 ('Seeking', 0.47591522157996147),
 ('Venus,"', 0.4639175257731959),
 ('pursuit', 0.4607679465776294),
 ('valuable.', 0.4318181818181818),
 ('presents.', 0.4274193548387097),
 ('Venus"', 0.42574257425742573),
 ('worthy', 0.41895261845386533),
 ('disagree,', 0.4189189189189189),
 ('Challenge', 0.39617486338797814),
 ('Exploring', 0.3825503355704698),
 ('advantages', 0.36929460580912865),
 ('why.', 0.35344827586206895),
 ('capacity', 0.35294117647058826),
 ('strongly', 0.34096692111959287),
 ('Studying', 0.3402061855670103),
 ('despite', 0.3392265193370166),
 ('senator', 0.32978723404255317),
 ('abolished.', 0.3194444444444444),
 ('periods.', 0.31666666666666665),
 ('\nI', 0.3125),
 ('Limiting', 0.3115264797507788),
 ('presents', 0.31140350877192985),
 ('disagree', 0.30686406460296095),
 ('beleive', 0.30097087378640774),
 ('dangers,', 0.3),
 ('Cowboys', 0.2935897435897436),
 ('reasons.', 0.2932692307692308),
 ('Policy', 0.

In [212]:
claim_counts_ratios.most_common()

[('Secondly,', 0.7567567567567568),
 ('Firstly,', 0.6741573033707865),
 ('Second', 0.6363636363636364),
 ('First', 0.6329896907216495),
 ('First,', 0.6275862068965518),
 ('Second,', 0.6176470588235294),
 ('begin,', 0.6101694915254238),
 ('Third,', 0.5873015873015873),
 ('Next,', 0.5743243243243243),
 ('Lastly,', 0.5224215246636771),
 ('certainty', 0.4397590361445783),
 ('Finally,', 0.4375),
 ('Another', 0.435929648241206),
 ('reason', 0.43383047007740233),
 ('\nThe', 0.42857142857142855),
 ('emergencies', 0.4264705882352941),
 ('Lastly', 0.4051724137931034),
 ('Last', 0.4030612244897959),
 ('responsibility,', 0.39344262295081966),
 ('Finally', 0.3828125),
 ('least,', 0.37777777777777777),
 ('disabilities', 0.37719298245614036),
 ('off,', 0.37656903765690375),
 ('lowers', 0.3728813559322034),
 ('disaster', 0.36789297658862874),
 ('factor.', 0.3602941176470588),
 ('future.\n', 0.3584905660377358),
 ('Astronomers', 0.35555555555555557),
 ('creativity.', 0.35365853658536583),
 ('voters.', 

In [213]:
counterclaim_counts_ratios.most_common()

[('argue', 0.5295454545454545),
 ('downside', 0.40350877192982454),
 ('markings', 0.3779527559055118),
 ('bona', 0.3548387096774194),
 ('hidden', 0.35185185185185186),
 ('fide', 0.3333333333333333),
 ('theorists', 0.28378378378378377),
 ('argued', 0.27941176470588236),
 ('Others', 0.2647058823529412),
 ('conspiracy', 0.2542372881355932),
 ('Posner,', 0.25),
 ('Sure', 0.2441860465116279),
 ('Yes,', 0.24233128834355827),
 ('downsides', 0.24193548387096775),
 ('hand,', 0.24),
 ('wispy', 0.23809523809523808),
 ('hiding', 0.2222222222222222),
 ('con', 0.22033898305084745),
 ('cloudy', 0.21774193548387097),
 ('counter', 0.2112676056338028),
 ('Yes', 0.19747899159663865),
 ('Although,', 0.19615384615384615),
 ('avoids', 0.19246861924686193),
 ('Although', 0.18955399061032863),
 ('Some', 0.17587419822056694),
 ('recieves', 0.1746031746031746),
 ('certainty', 0.1686746987951807),
 ('though.', 0.1643835616438356),
 ('understandable', 0.15789473684210525),
 ('hands.', 0.1523809523809524),
 ('disp

In [214]:
rebuttal_counts_ratios.most_common()

[('true,', 0.32298136645962733),
 ('defenders', 0.2962962962962963),
 ('incorrect', 0.2698412698412698),
 ('budget', 0.22321428571428573),
 ('flawed', 0.1917808219178082),
 ('However,', 0.1782343142698501),
 ('civilization', 0.16666666666666666),
 ('cloudless', 0.16666666666666666),
 ('drew', 0.16666666666666666),
 ('But,', 0.15104166666666666),
 ('however,', 0.14985163204747776),
 ('reality,', 0.14655172413793102),
 ('ancient', 0.1404494382022472),
 ('2001,', 0.13829787234042554),
 ('8,', 0.13740458015267176),
 ('Yet,', 0.1320754716981132),
 ('however', 0.12941176470588237),
 ('hazardous', 0.12857142857142856),
 ('case.', 0.1259259259259259),
 ('valid', 0.12422360248447205),
 ('true', 0.1225296442687747),
 ('true.', 0.1222707423580786),
 ('However', 0.1167192429022082),
 ('necessarily', 0.11607142857142858),
 ('outweigh', 0.11347517730496454),
 ('disagree.', 0.11320754716981132),
 ('downsides', 0.11290322580645161),
 ('interaction.', 0.10909090909090909),
 ('but,', 0.10816777041942605

In [215]:
evidence_counts_ratios.most_common()

[('odd-numbered', 0.9908256880733946),
 ('even-numbered', 0.9882352941176471),
 ('Diesel', 0.9878048780487805),
 ('skated', 0.9868421052631579),
 ('micrograms', 0.9861111111111112),
 ('118', 0.9857142857142858),
 ('particulate', 0.9848484848484849),
 ('numbered', 0.9846153846153847),
 ('Rosenthal,', 0.9838709677419355),
 ('147', 0.9838709677419355),
 ('sediment', 0.983739837398374),
 ('gondola', 0.9819819819819819),
 ('frown,', 0.9814814814814815),
 ('57', 0.9811320754716981),
 ('Louisiana', 0.9809523809523809),
 ('Kennedy.', 0.9805825242718447),
 ('1960,', 0.9803921568627451),
 ('motorists', 0.979381443298969),
 ('Kennedy', 0.979381443298969),
 ('biked,', 0.9789473684210527),
 ('22-euro', 0.9775280898876404),
 ('plates', 0.9765395894428153),
 ('F.', 0.9760479041916168),
 ('Romney', 0.9747899159663865),
 ('67', 0.9726027397260274),
 ('Toyota', 0.9710144927536232),
 ('Earth."', 0.9696969696969697),
 ('cubic', 0.9696969696969697),
 ('intensifying', 0.9693877551020408),
 ('capital', 0.969

In [216]:
conclustate_counts_ratios.most_common()

[('\xa0\xa0', 0.9386973180076629),
 ('conclusion,', 0.9120961682945155),
 ('Thank', 0.8961038961038961),
 ('\xa0', 0.892899268695447),
 ('conclude,', 0.8407079646017699),
 ('Please', 0.7757009345794392),
 ('conclusion', 0.7744063324538258),
 ('Overall,', 0.6228571428571429),
 ('Overall', 0.6181818181818182),
 ('sum', 0.6019417475728155),
 ('\nIn', 0.6),
 ('conclude', 0.5984251968503937),
 ('hope', 0.5902992776057792),
 ('please', 0.5486111111111112),
 ('Hopefully', 0.5357142857142857),
 ('Remember', 0.5333333333333333),
 ('lastly', 0.5058823529411764),
 ('consideration.', 0.4931506849315068),
 ('views,', 0.4918032786885246),
 ('creative,', 0.47619047619047616),
 ('end,', 0.45320197044334976),
 ('Ultimately,', 0.4482758620689655),
 ('sake', 0.4426229508196721),
 ('safer,', 0.43820224719101125),
 ('Always', 0.43478260869565216),
 ('outweigh', 0.4326241134751773),
 ('thank', 0.432),
 ('increase.', 0.4117647058823529),
 ('safety,', 0.4050632911392405),
 ('come.', 0.40458015267175573),
 ('i

In [217]:
conclustate_counts.most_common()

[('the', 33674),
 ('to', 29959),
 ('and', 23042),
 ('', 22679),
 ('a', 18559),
 ('of', 14257),
 ('is', 12898),
 ('that', 11911),
 ('be', 11842),
 ('you', 11193),
 ('in', 10465),
 ('it', 10000),
 ('for', 9019),
 ('are', 7406),
 ('they', 7328),
 ('have', 7075),
 ('can', 7049),
 ('not', 6855),
 ('students', 6564),
 ('I', 5997),
 ('will', 5766),
 ('should', 5754),
 ('would', 5535),
 ('on', 5425),
 ('more', 5251),
 ('people', 5122),
 ('their', 4884),
 ('because', 4340),
 ('or', 4335),
 ('with', 4225),
 ('we', 4163),
 ('this', 4105),
 ('\xa0', 3785),
 ('do', 3778),
 ('help', 3749),
 ('make', 3532),
 ('but', 3423),
 ('if', 3250),
 ('from', 3201),
 ('as', 3173),
 ('think', 3109),
 ('all', 3098),
 ('The', 3085),
 ('your', 3059),
 ('just', 2993),
 ('school', 2973),
 ('get', 2919),
 ('what', 2842),
 ('could', 2783),
 ('at', 2672),
 ('better', 2611),
 ('one', 2513),
 ('them', 2496),
 ('In', 2490),
 ('many', 2459),
 ('our', 2392),
 ('there', 2387),
 ('by', 2379),
 ('about', 2345),
 ('an', 2282),
 (